In [1]:
import pandas as pd
import os
import datetime
import re
import numpy as np
import graphlab as gl
from subprocess import check_output
os.chdir('/Users/sobh/Desktop/abby/go2git/titanic_data')

In [2]:
df = pd.read_csv('train.csv', header=0)
cf = pd.read_csv('test.csv', header=0)

In [3]:
#df['Name'].unique()
#sum(df['Cabin'].isnull())

In [4]:
def formatDF(df):
    df['Gender'] = df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    df = df.drop(['Sex'], axis=1)

    # Since the missing embarked values are both from 1st class tickets,
    # can use the ticket price to infer they are embarking from 'C'
    df['Embarked'] = df['Embarked'].map( {np.nan: 2, 'S': 1, 'C': 2,\
                                            'Q': 3} ).astype(int)
   
    df['AgeMedians'] = df['Age']
    df['AgeMeans'] = df['Age']
    df['FareMedians'] = df['Fare']
    df['FareMeans'] = df['Fare']

    
    df2 = df
    
    df2['Cabin'] = df2['Cabin'].str.replace('.*A.*','A')
    df2['Cabin'] = df2['Cabin'].str.replace('.*B.*','B')
    df2['Cabin'] = df2['Cabin'].str.replace('.*C.*','C')
    df2['Cabin'] = df2['Cabin'].str.replace('.*D.*','D')
    df2['Cabin'] = df2['Cabin'].str.replace('.*E.*','E')
    df2['Cabin'] = df2['Cabin'].str.replace('.*F.*','F')
    df2['Cabin'] = df2['Cabin'].str.replace('.*G.*','G')
    df2['Cabin'] = df2['Cabin'].str.replace('.*T.*','T')    
    df2.loc[ (df2.Cabin.isnull()),\
                    'Cabin'] = 'NoCabin'
    

    df2['LastName'] = df['Name'].map(lambda x: x.split(',')[0])
    df2['Name'] = df2['Name'].str.replace('.*Mr.*','1')
    df2['Name'] = df2['Name'].str.replace('.*Jonkheer.*','8')
    df2['Name'] = df2['Name'].str.replace('.*Countess.*','8')
    df2['Name'] = df2['Name'].str.replace('.*Don.*','1')
    df2['Name'] = df2['Name'].str.replace('.*Major.*','4')
    df2['Name'] = df2['Name'].str.replace('.*Col.*','4')
    df2['Name'] = df2['Name'].str.replace('.*Capt.*','4')
    df2['Name'] = df2['Name'].str.replace('.*Miss.*','3')
    df2['Name'] = df2['Name'].str.replace('.*Ms.*','3')
    df2['Name'] = df2['Name'].str.replace('.*Mlle.*','3')
    df2['Name'] = df2['Name'].str.replace('.*Mrs.*','2')
    df2['Name'] = df2['Name'].str.replace('.*Mme.*','2')
    df2['Name'] = df2['Name'].str.replace('.*Master.*','5')
    df2['Name'] = df2['Name'].str.replace('.*Rev.*','6')
    df2['Name'] = df2['Name'].str.replace('.*Dr.*','7')
    df['Name'] = df2['Name']


    median_ages = np.zeros((2,3))
    mean_ages = np.zeros((2,3))
    
    median_fares = np.zeros((3,3))
    mean_fares = np.zeros((3,3))
    
    for i in range(0, 2):
        for j in range(0, 3):
            median_ages[i,j] = df[(df['Gender'] == i) & \
                                  (df['Pclass'] == j+1)]['Age'].dropna().median()
            mean_ages[i,j] = df[(df['Gender'] == i) & \
                                  (df['Pclass'] == j+1)]['Age'].dropna().mean()     

    for i in range(0, 3):
        for j in range(0, 3):            
            median_fares[i,j] = df[(df['Embarked'] == i) & \
                                  (df['Pclass'] == j+1)]['Fare'].dropna().median()
            mean_fares[i,j] = df[(df['Embarked'] == i) & \
                                  (df['Pclass'] == j+1)]['Fare'].dropna().mean()   

    for i in range(0, 2):
        for j in range(0, 3):
            df.loc[ (df.Age.isnull()) & (df.Gender == i) & (df.Pclass == j+1),\
                    'AgeMedians'] = median_ages[i,j]
            df.loc[ (df.Age.isnull()) & (df.Gender == i) & (df.Pclass == j+1),\
                    'AgeMeans'] = mean_ages[i,j]      
            
    for i in range(0, 3):
        for j in range(0, 3):            
            df.loc[ (df.Fare.isnull()) & (df.Embarked == i) & (df.Pclass == j+1),\
                    'FareMedians'] = median_fares[i,j]
            df.loc[ (df.Fare.isnull()) & (df.Embarked == i) & (df.Pclass == j+1),\
                    'FareMeans'] = mean_fares[i,j]                
    return df

In [5]:
df = formatDF(df)
dftwo = df

In [6]:
cf = formatDF(cf)
dfthree = cf

In [7]:
# Binning the fares
def functionb(data):
    if (data < 8):
        return 0
    elif (8<=data<15):
        return 1
    elif (15<=data<32):
        return 2
    elif (data >= 32):
        return 3
    else:
        return 1

dftwo['AdjFare'] = dftwo['Fare'].map(functionb)
dfthree['AdjFare'] = dfthree['Fare'].map(functionb)
df2 = dftwo
df3 = dfthree

In [8]:
# add the FamilySize, FamilyTier parameters
dftwo['FamilySize'] = dftwo['SibSp'] + dftwo['Parch']
dfthree['FamilySize'] = dfthree['SibSp'] + dfthree['Parch']
def functionc(data):
    if (data == 0):
        return 1
    elif (data > 4):
        return 0
    else:
        return 2

dftwo['FamilyTier'] = dftwo['FamilySize'].map(functionc)
dfthree['FamilyTier'] = dfthree['FamilySize'].map(functionc)

In [9]:
# If multiple people share the same ticket or only a single person on the ticket
def functionX2(data):
    ticket_count = dftwo['Ticket'].value_counts()[data]
    if (ticket_count > 1):
        return 1
    else:
        return 0
    
def functionX3(data):
    ticket_count = dfthree['Ticket'].value_counts()[data]
    if (ticket_count > 1):
        return 1
    else:
        return 0

dftwo['HasTicketMates'] = dftwo['Ticket'].map(functionX2)
dfthree['HasTicketMates'] = dfthree['Ticket'].map(functionX3)



In [10]:
# The Train Set
df2 = dftwo
# The Test Set
df3 = dfthree

In [11]:
prTrainFull = gl.SFrame(data=df2)
prTrain = prTrainFull.dropna()
prTestFull  = gl.SFrame(data=df3)
prTest = prTestFull.dropna()
# Make a train-test split

train_data, test_data = (prTrain).random_split(0.8)
# Create a model.
# gl.boosted_trees_classifier.create
# gl.classifier.create
model = gl.boosted_trees_regression.create(train_data, target='Age',
                            features=['AgeMedians','AgeMeans', 'Gender'])



df2['PredictedAge'] = model.predict(prTrainFull)
df3['PredictedAge'] = model.predict(prTestFull)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1474344828.log


This non-commercial license of GraphLab Create for academic use is assigned to abbysobh@gmail.com and will expire on July 13, 2017.
PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Boosted trees regression:

--------------------------------------------------------

Number of examples          : 555

...

| 3         | 0.019318     | 40.342545          | 24.342545            | 11.471787     | 11.756824       |

| 4         | 0.025206     | 32.926769          | 16.926769            | 8.145508      | 8.235658        |

| 5         | 0.029265     | 26.824013          | 10.824013            | 5.793760      | 5.589430        |

| 6         | 0.033017     | 21.938957          | 7.971039             | 4.133640      | 3.989734        |

+-----------+--------------+--------------------+----------------------+---------------+-----------------+

In [12]:
def functiond(data):
    if (data > 16):
        return 1
    else:
        return 0
    
df2['IsAdult'] = df2['PredictedAge'].map(functiond)
df3['IsAdult'] = df3['PredictedAge'].map(functiond)


# Credit to Omar El Gabry (https://www.kaggle.com/omarelgabry)
# For treating children as a non-gender.
# This helps to slightly unskew the Gender/Survival bias.
df2.loc[ (df2.IsAdult == 0, 'Gender')] = 2
df3.loc[ (df3.IsAdult == 0, 'Gender')] = 2    


In [22]:
# diagnostic...should be at the top of this code, not here
#for item in df2.columns:
#    if item not in ("PassengerId","Age","Ticket"):
#        print(item)
#        print(df2[item].value_counts())

In [13]:
shelTrain0 = gl.SFrame(data=df2)
#dropTrain = df4
shelTest0 = gl.SFrame(data=df3)


In [14]:
# Create a model.
# gl.boosted_trees_classifier.create
# gl.classifier.create

train_data,test_data = shelTrain0.random_split(0.8)
model = gl.boosted_trees_regression.create(train_data, target='Survived',
                                           features=[

 'Pclass',
 'Name',
 'LastName',
# 'Age',
# 'SibSp',
# 'Parch',
# 'Ticket',
# 'Fare',
 'Cabin',
 'Embarked',
 'Gender',
# 'AgeMedians',
# 'AgeMeans',
# 'FareMedians',
# 'FareMeans',
 'AdjFare',
 'FamilySize',
 'FamilyTier',
 'HasTicketMates',
 'PredictedAge',
 'IsAdult'
                                                    ], max_iterations=300)

predictions = model.predict(test_data)
# Evaluate the model and save the results into a dictionary
#results = model.evaluate(test_data)
#results

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Boosted trees regression:

--------------------------------------------------------

Number of examples          : 680

Number of features          : 12

...

| 11        | 0.066959     | 0.896931           | 0.957379             | 0.261038      | 0.332487        |

| 51        | 0.215394     | 0.943196           | 0.943196             | 0.187128      | 0.334420        |

| 101       | 0.385766     | 0.649909           | 0.965607             | 0.136942      | 0.339923        |

| 250       | 0.966840     | 0.557959           | 0.987718             | 0.064452      | 0.363315        |

+-----------+--------------+--------------------+----------------------+---------------+-----------------+

In [16]:
#shelTrain0.column_names()
#model.get_feature_importance()
#model.get('coefficients')

In [17]:
#model.show()
#model.show(view="Tree", tree_id=0)

In [18]:
predictions = model.predict(test_data)
# Evaluate the model and save the results into a dictionary
results = model.evaluate(test_data)
results

{'max_error': 1.2462415099143982, 'rmse': 0.40818917457607884}

In [516]:
save_model = model
type(save_model)

graphlab.toolkits.regression.boosted_trees_regression.BoostedTreesRegression

In [19]:
# A model comparison maker between a certain model and future iterations...as a function
# For regression models...
import math
def log_loss_raw(target, predicted):
    '''Calculate log_loss between target and predicted and return.'''
    p = predicted.apply(lambda x: min(0.99999, max(1e-5, x)))
    logp = p.apply(lambda x: math.log(x))
    logmp = p.apply(lambda x: (math.log(1-x)))
    return -(target * logp + (1-target) * logmp).mean()

def eval_model(model, test):
    '''Evaluate a trained model using Kaggle scoring.'''
    return log_loss_raw(test['Survived'], model.predict(test))

# Usage:
# print "%0.20f" % eval_model(baseline_model, test_set)

In [20]:
print "%0.20f" % eval_model(model, test_data)

0.68159369018371873672


In [529]:
#drop_ages_predict = model.predict(shelTest)
#shelTest['PredictedAge'] = drop_ages_predict
#shelTrain

In [236]:
#shelTest.export_csv('Predicted_Test.csv')
#shelTrain.export_csv('Predicted_Train.csv')
#shelTest

In [21]:
#true_predictions = model.predict(hts)
#true_predictions = model.predict(qsg)
true_predictions = model.predict(shelTest0)

In [22]:
# Basic rounding
a = true_predictions.apply(lambda x: (x>=0.5))

In [23]:
# If used regression, round the values at a cutoff
a = true_predictions.apply(lambda x: (x>0.6))
b = true_predictions.apply(lambda x: (x>=0.5))

#true_predictions
sum(a == b)
#b

406

In [24]:
#shelTest0['Survived'] = true_predictions
shelTest0['Survived'] = a

In [25]:
shelTestSubmit = shelTest0['PassengerId','Survived']

In [55]:
shelTestSubmit.save('Titanic_Results.csv')